In [1]:
import pandas as pd
import numpy as np
import os
import torch
from torch.utils.data import Dataset
import nibabel as nib
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import glob
from torchvision import transforms

In [2]:
df = pd.read_excel('all_patients.xlsx')
df = df[df.shape_VoxelVolume > 125]
df.drop(df[df['labels'] == 3].index, inplace = True) ## dropping label of the call OTHERS
df.drop(df[df['number_annotations'] > 1].index, inplace = True)
df

,subjid,label,shape_Elongation,shape_Flatness,shape_LeastAxisLength,shape_MajorAxisLength,shape_Maximum2DDiameterColumn,shape_Maximum2DDiameterRow,shape_Maximum2DDiameterSlice,shape_Maximum3DDiameter,...,ngtdm_Busyness,ngtdm_Coarseness,ngtdm_Complexity,ngtdm_Contrast,ngtdm_Strength,Age,number_annotations,F,M,labels
0,Oslo01,1,0.829356,0.682981,8.512717,12.464063,14.866069,14.560220,12.529964,15.394804,...,158.977235,0.008929,0.302273,0.074659,0.008950,58,1,0,1,0
1,Oslo01,2,0.944143,0.900681,14.133927,15.692487,17.804494,19.416488,18.681542,19.467922,...,39.445794,0.007407,0.115448,0.010426,0.007130,58,1,0,1,0
4,Oslo02,4,0.849295,0.841082,8.703783,10.348315,10.816654,11.704700,11.704700,12.449900,...,14.239428,0.022868,0.159307,0.020316,0.023465,50,1,1,0,1
5,Oslo02,5,0.939828,0.821847,12.093804,14.715396,17.262677,17.888544,17.492856,18.000000,...,49.609354,0.008536,0.137901,0.028455,0.008212,50,1,1,0,1
7,Oslo03,1,0.930698,0.629604,10.914986,17.336268,20.615528,18.439089,19.416488,20.712315,...,43.648474,0.007781,0.123220,0.017415,0.007982,64,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,Stan_328,1,0.810571,0.325076,8.769420,26.976501,31.384710,28.160256,27.459060,33.075671,...,12.988427,0.019891,0.030104,0.000639,0.019768,53,1,0,1,1
2415,Stan_328,8,0.967496,0.597282,6.131668,10.265944,11.661904,12.369317,12.165525,12.688578,...,18.452502,0.024791,0.208465,0.046975,0.023205,53,1,0,1,1
2418,Stan_328,11,0.561746,0.506390,5.004740,9.883181,8.246211,11.704700,10.000000,12.124356,...,0.553323,0.468447,0.025263,0.000748,0.365681,53,1,0,1,1
2419,Stan_328,12,0.959775,0.756903,5.164755,6.823538,7.615773,8.602325,8.602325,8.602325,...,0.000000,1000000.000000,0.000000,0.000000,0.000000,53,1,0,1,1


In [3]:
new_df = pd.DataFrame({'Subjid': df.iloc[:, 0], 'Tumor': df.iloc[:, 1], 'Labels': df.iloc[:, -1]})
new_df = new_df.drop_duplicates(subset='Subjid', keep='first')
#new_df.to_excel('labels_DL.xlsx', index = False)
new_df

,Subjid,Tumor,Labels
0,Oslo01,1,0
4,Oslo02,4,1
7,Oslo03,1,0
11,Oslo04,1,0
13,Oslo06,1,1
...,...,...,...
2359,Stan_318,1,1
2369,Stan_319,2,1
2387,Stan_323,3,1
2408,Stan_328,1,1


In [4]:
stan = new_df['Subjid'][59:].to_list()
oslo = new_df['Subjid'][:59].to_list()

## we have created a new folder called deep learning with all the folder with all the folder per patient labels as 0, 1 or 2 with tumors greater than 125mm3

In [324]:
# Specify the path to the parent folder
parent_folder = '/data/projects/TMOR/data/Deeplearning/'

# Get a list of all subfolders within the parent folder
subfolders = [f.path for f in os.scandir(parent_folder) if f.is_dir()]

# Iterate over the subfolders and rename them

for folder in subfolders:
    folder_name = os.path.basename(folder)
    new_folder_name = folder_name.replace("Subject", "Oslo")
    #new_folder_name = folder_name.replace("Mets", "Stan")
    new_folder_path = os.path.join(parent_folder, new_folder_name)
    os.rename(folder, new_folder_path)


In [325]:
#delete the other file for OSLO
directory = '/data/projects/TMOR/data/Deeplearning/'

# Get a list of all patient folders
patient_folders = glob.glob(os.path.join(directory, 'Oslo*'))

for patient_folder in patient_folders:
    #id = patient_folder[-2:]
    seg_folders = glob.glob(os.path.join(patient_folder, 'seg*'))
    t1_gds = glob.glob(os.path.join(patient_folder, 't1_gd*'))
    #print(seg_folders)
    # Iterate over each seg folder
    for seg_folder in seg_folders:
        for t1_gd in t1_gds:
            #print(seg_folder)
            image_files = glob.glob(os.path.join(patient_folder, '*'))
            #print(image_files)
            # Delete all the image files except for 'seg_1'
            for file in image_files:
            # Check if the file is in the segmentation folder or T1-Gd folder
                if file not in [seg_folder, t1_gd]:
                    os.remove(file)

In [326]:
patient_folders = glob.glob(os.path.join(directory, 'Oslo*'))
import shutil
for patient_folder in patient_folders:
    id = patient_folder[-6:]
    if id not in oslo:
        shutil.rmtree(patient_folder)

In [196]:
import os

directory = '/data/projects/TMOR/data/Deeplearning/'

patient_folders = glob.glob(os.path.join(directory, 'Oslo*'))


# Iterate over patient folders
for patient_folder in patient_folders:
    #patient_path = os.path.join(directory, patient_folder)
    seg_folders = glob.glob(os.path.join(patient_folder, 'seg*'))
    # Rename segmentation files
    for seg_folder in seg_folders:
        seg_old_path = seg_folder
        seg_new_path = os.path.join(patient_folder, 'seg.nii.gz')
        os.rename(seg_old_path, seg_new_path)
        #print(seg_new_path)

## STANDFORD DATA

In [328]:
# Specify the path to the parent folder
parent_folder = '/data/projects/TMOR/data/Deeplearning/'

# Get a list of all subfolders within the parent folder
subfolders = [f.path for f in os.scandir(parent_folder) if f.is_dir()]

# Iterate over the subfolders and rename them

for folder in subfolders:
    folder_name = os.path.basename(folder)
    new_folder_name = folder_name.replace("Mets", "Stan")
    new_folder_path = os.path.join(parent_folder, new_folder_name)
    os.rename(folder, new_folder_path)


In [329]:
import os
import glob
#delete the other file for OSLO
directory = '/data/projects/TMOR/data/Deeplearning/'

# Get a list of all patient folders
patient_folders = glob.glob(os.path.join(directory, 'Stan*'))

for patient_folder in patient_folders:
    seg_folders = glob.glob(os.path.join(patient_folder, 'seg*'))
    t1_gds = glob.glob(os.path.join(patient_folder, 't1_gd*'))
    # Iterate over each seg folder
    for seg_folder in seg_folders:
        for t1_gd in t1_gds:
            image_files = glob.glob(os.path.join(patient_folder, '*'))
            # Delete all the image files except for 'seg_1'
            for file in image_files:
                # Check if the file is in the segmentation folder or T1-Gd folder
                if file not in [seg_folder, t1_gd]:
                    os.remove(file)


In [330]:
#Remove all the files that are not within stan
import shutil
for patient_folder in patient_folders:
    id = patient_folder[-8:]
    if id not in stan:
        shutil.rmtree(patient_folder)

## move all t1_pre to the right folder of /data/projects/TMOR/data/Deeplearning/A/

## OSLO

In [28]:
# Specify the path to the parent folder
parent_folder = '/data/projects/TMOR/data/OsloPreprocessed/OsloPreprocessed/'

# Get a list of all subfolders within the parent folder
subfolders = [f.path for f in os.scandir(parent_folder) if f.is_dir()]

# Iterate over the subfolders and rename them

for folder in subfolders:
    folder_name = os.path.basename(folder)
    new_folder_name = folder_name.replace("Subject", "Oslo")
    #new_folder_name = folder_name.replace("Mets", "Stan")
    new_folder_path = os.path.join(parent_folder, new_folder_name)
    os.rename(folder, new_folder_path)

In [44]:
import shutil
directory = '/data/projects/TMOR/data/OsloPreprocessed/OsloPreprocessed/'

patient_folders = glob.glob(os.path.join(directory, 'Oslo*'))

repository = '/data/projects/TMOR/data/Deeplearning/'

pacientes = glob.glob(os.path.join(repository, 'Oslo*'))

for patient_folder in patient_folders:
    t1_pre_files = glob.glob(os.path.join(patient_folder, 't1_pre*'))
    for t1 in t1_pre_files:
        patient_id = os.path.basename(patient_folder)
        patient_match = next((p for p in pacientes if patient_id in p), None)
        if patient_match:
        # Move each t1-pre file to the respective patient folder
            for t1_pre_file in t1_pre_files:
                destination_folder = os.path.join(patient_match, os.path.basename(t1_pre_file))
                shutil.copy(t1_pre_file, destination_folder)

## Standford

In [45]:
# Specify the path to the parent folder
parent_folder = '/data/projects/TMOR/data/StanfordPreprocessed/StanfordPreprocessed/'

# Get a list of all subfolders within the parent folder
subfolders = [f.path for f in os.scandir(parent_folder) if f.is_dir()]

# Iterate over the subfolders and rename them

for folder in subfolders:
    folder_name = os.path.basename(folder)
    #new_folder_name = folder_name.replace("Subject", "Oslo")
    new_folder_name = folder_name.replace("Mets", "Stan")
    new_folder_path = os.path.join(parent_folder, new_folder_name)
    os.rename(folder, new_folder_path)

In [46]:
directory = '/data/projects/TMOR/data/StanfordPreprocessed/StanfordPreprocessed/'

patient_folders = glob.glob(os.path.join(directory, 'Stan*'))

repository = '/data/projects/TMOR/data/Deeplearning/'

pacientes = glob.glob(os.path.join(repository, 'Stan*'))

for patient_folder in patient_folders:
    t1_pre_files = glob.glob(os.path.join(patient_folder, 't1_pre*'))
    for t1 in t1_pre_files:
        patient_id = os.path.basename(patient_folder)
        patient_match = next((p for p in pacientes if patient_id in p), None)
        if patient_match:
        # Move each t1-pre file to the respective patient folder
            for t1_pre_file in t1_pre_files:
                destination_folder = os.path.join(patient_match, os.path.basename(t1_pre_file))
                shutil.copy(t1_pre_file, destination_folder)

## class object

## lets divide the tumors

In [575]:
D_x_max, D_y_max, D_z_max = 0, 0, 0
seg_dir = '/data/projects/TMOR/data/Deeplearning/'
#indices = []
for subj_id in tqdm(os.listdir(seg_dir)):
    #extrasct the correct id from the subjects
    # Define paths to the segmentation and regular t1 postprocessed files for this subject
    seg_glob = os.path.join(seg_dir, f'{subj_id}', f'seg.nii.gz')
    reg_glob = os.path.join(seg_dir, f'{subj_id}', f't1_gd.nii.gz')
    seg_paths = glob.glob(seg_glob)
    reg_paths = glob.glob(reg_glob)
    # Loop through all matching segmentation and registration files
    for seg, reg in zip(seg_paths, reg_paths):
        if os.path.exists(seg) and os.path.exists(reg):
            mask = nib.load(seg)
            vol_mask = mask.get_fdata()
            for label in np.unique(vol_mask):
                if label != 0:
                    label_indices = np.argwhere(vol_mask == label)
                    if len(label_indices) > 125:
                        indices = label_indices
                        X_min, X_max = np.min(indices[:, 0]), np.max(indices[:, 0])
                        Y_min, Y_max = np.min(indices[:, 1]), np.max(indices[:, 1])
                        Z_min, Z_max = np.min(indices[:, 2]), np.max(indices[:, 2])
                        #we need the differences now boi
                        D_x = X_max - X_min
                        D_y = Y_max - Y_min
                        D_z = Z_max - Z_min
                        if D_x_max < D_x:
                        	D_x_max = D_x
                        if D_y_max < D_y:
                        	D_y_max = D_y
                        if D_z_max < D_z:
                        	D_z_max = D_z

100%|██████████| 157/157 [00:50<00:00,  3.09it/s]


## very final A

In [576]:
drop_dir = '/data/projects/TMOR/data/'
seg_dir = '/data/projects/TMOR/data/Deeplearning/'
#indices = []
from tqdm import tqdm
for subj_id in tqdm(os.listdir(seg_dir)):
    #extrasct the correct id from the subjects
    # Define paths to the segmentation and regular t1 postprocessed files for this subject
    seg_glob = os.path.join(seg_dir, f'{subj_id}', f'seg.nii.gz')
    reg_glob = os.path.join(seg_dir, f'{subj_id}', f't1_gd.nii.gz')
    reg2_glob = os.path.join(seg_dir, f'{subj_id}', f't1_pre.nii.gz')
    seg_paths = glob.glob(seg_glob)
    reg_paths = glob.glob(reg_glob)
    reg2_paths = glob.glob(reg2_glob)
    # Loop through all matching segmentation and registration files
    for seg, reg, reg2 in zip(seg_paths, reg_paths, reg2_paths):
        if os.path.exists(seg) and os.path.exists(reg) and os.path.exists(reg2):
            mask = nib.load(seg)
            vol_mask = mask.get_fdata()
            t1_gd = nib.load(reg)
            vol_t1_gd = t1_gd.get_fdata()
            t1_pre = nib.load(reg2)
            vol_t1_pre = t1_pre.get_fdata()
            vol = vol_t1_gd.shape
            for label in np.unique(vol_mask):
                if label != 0:
                    label_indices = np.argwhere(vol_mask == label)
                    if len(label_indices) > 125:
                        indices = label_indices
                        X_min, X_max = np.min(indices[:, 0]), np.max(indices[:, 0])
                        Y_min, Y_max = np.min(indices[:, 1]), np.max(indices[:, 1])
                        Z_min, Z_max = np.min(indices[:, 2]), np.max(indices[:, 2])

                        X_avg = (X_min + X_max) // 2
                        Y_avg = (Y_min + Y_max) // 2
                        Z_avg = (Z_min + Z_max) // 2
                        # Pad the indices with zeros

                        X_low = max(X_avg - (D_x_max // 2), 0)
                        X_high = min(X_avg + (D_x_max // 2),  vol_mask.shape[0])
                        Y_low = max(Y_avg - (D_y_max // 2), 0)
                        Y_high = min(Y_avg + (D_y_max // 2), vol_mask.shape[1])
                        Z_low = max(Z_avg - (D_z_max // 2), 0)
                        Z_high = min(Z_avg + (D_z_max // 2), vol_mask.shape[2])

                        pene = vol_mask[X_low:X_high, Y_low:Y_high, Z_low:Z_high]
                        t1_desp = vol_t1_gd[X_low:X_high, Y_low:Y_high, Z_low:Z_high]
                        t1_antes = vol_t1_pre[X_low:X_high, Y_low:Y_high, Z_low:Z_high]

                        ## define the pads
                        pad_width = [
                            (0, max(D_x_max - pene.shape[0], 0)),
                            (0, max(D_y_max - pene.shape[1], 0)),
                            (0, max(D_z_max - pene.shape[2], 0))
                        ]
                        # define the partitions
                        seg_partitioned = np.pad(pene, pad_width, mode='constant', constant_values=0)
                        t1_gd_partitioned = np.pad(t1_desp, pad_width, mode='constant', constant_values=0)
                        t1_pre_partitioned = np.pad(t1_antes, pad_width, mode='constant', constant_values=0)
                        #
                        output_dir = os.path.join(drop_dir, 'VeryFinalA', subj_id, str(int(label)))
                        os.makedirs(output_dir, exist_ok=True)
                        # tumors segemented of the segmentation of mask
                        output_path_1 = os.path.join(output_dir, 'seg.nii.gz')
                        nib.save(nib.Nifti1Image(seg_partitioned, mask.affine), output_path_1)
                        # tumors segmented of the t1_gd
                        output_path_2 = os.path.join(output_dir, 't1_gd.nii.gz')
                        nib.save(nib.Nifti1Image(t1_gd_partitioned, t1_gd.affine), output_path_2)
                        # tumors segemented if t1_pre
                        output_path_3 = os.path.join(output_dir, 't1_pre.nii.gz')
                        nib.save(nib.Nifti1Image(t1_pre_partitioned, t1_pre.affine), output_path_3)

100%|██████████| 157/157 [01:36<00:00,  1.62it/s]


In [593]:
from matplotlib import pyplot as plt

drop_dir = '/data/projects/TMOR/data/'
seg_dir = '/data/projects/TMOR/data/Deeplearning/'
#indices = []
for subj_id in tqdm(os.listdir(seg_dir)):
    #extrasct the correct id from the subjects
    # Define paths to the segmentation and regular t1 postprocessed files for this subject
    seg_glob = os.path.join(seg_dir, f'{subj_id}', f'seg.nii.gz')
    reg_glob = os.path.join(seg_dir, f'{subj_id}', f't1_gd.nii.gz')
    reg2_glob = os.path.join(seg_dir, f'{subj_id}', f't1_pre.nii.gz')
    seg_paths = glob.glob(seg_glob)
    reg_paths = glob.glob(reg_glob)
    reg2_paths = glob.glob(reg2_glob)
    # Loop through all matching segmentation and registration files
    for seg, reg, reg2 in zip(seg_paths, reg_paths, reg2_paths):
        if os.path.exists(seg) and os.path.exists(reg) and os.path.exists(reg2):
            mask = nib.load(seg)
            vol_mask = mask.get_fdata()
            t1_gd = nib.load(reg)
            vol_t1_gd = t1_gd.get_fdata()
            t1_pre = nib.load(reg2)
            vol_t1_pre = t1_pre.get_fdata()
            vol = vol_t1_gd
            for label in np.unique(vol_mask):
                if label != 0:
                    label_indices = np.argwhere(vol_mask == label)
                    if len(label_indices) > 125:
                        indices = label_indices
                        X_min, X_max = np.min(indices[:, 0]), np.max(indices[:, 0])
                        Y_min, Y_max = np.min(indices[:, 1]), np.max(indices[:, 1])
                        Z_min, Z_max = np.min(indices[:, 2]), np.max(indices[:, 2])

                        mean_x = (X_min + X_max) // 2 #if != vol.shape[0] else vol.shape[0] - 1
                        mean_y = (Y_min + Y_max) // 2
                        mean_z  = (Z_min + Z_max) // 2

                        x_start = max(mean_x - (D_x_max // 2), 0)
                        x_end = min(mean_x + (D_x_max // 2),  vol_mask.shape[0])
                        y_start = max(mean_y - (D_y_max // 2), 0)
                        y_end = min(mean_y + (D_y_max // 2), vol_mask.shape[1])
                        z_start = max(mean_z - (D_z_max // 2), 0)
                        z_end = min(mean_z + (D_z_max // 2), vol_mask.shape[2])

                        fig, axes = plt.subplots(2,2)
                        axes[0,1].imshow(vol[mean_x, :, :].T, origin='lower')
                        axes[0,1].scatter(mean_y, mean_z)
                        axes[0,1].plot([y_start, y_start], [z_start, z_end])
                        axes[0,1].plot([y_end, y_end], [z_start, z_end])
                        axes[0,1].plot([y_start, y_end], [z_start, z_start])
                        axes[0,1].plot([y_start, y_end], [z_end, z_end])
                        axes[0,1].set_xlabel('y axis')
                        axes[0,1].set_ylabel('z axis')
                        axes[0,1].set_title('x plane')
                        axes[0,1].set_aspect('equal')
                        axes[0,0].imshow(vol[:, mean_y, :].T, origin='lower')
                        axes[0,0].scatter(mean_x, mean_z)
                        axes[0,0].plot([x_start, x_start], [z_start, z_end])
                        axes[0,0].plot([x_end, x_end], [z_start, z_end])
                        axes[0,0].plot([x_start, x_end], [z_start, z_start])
                        axes[0,0].plot([x_start, x_end], [z_end, z_end])
                        axes[0,0].set_xlabel('x axis')
                        axes[0,0].set_ylabel('z axis')
                        axes[0,0].set_title('y plane')
                        axes[0,0].set_aspect('equal')
                        axes[1,0].imshow(vol[:, :, mean_z].T, origin='lower')
                        axes[1,0].scatter(mean_x, mean_y)
                        axes[1,0].plot([x_start, x_start], [y_start, y_end])
                        axes[1,0].plot([x_end, x_end], [y_start, y_end])
                        axes[1,0].plot([x_start, x_end], [y_start, y_start])
                        axes[1,0].plot([x_start, x_end], [y_end, y_end])
                        axes[1,0].set_xlabel('x axis')
                        axes[1,0].set_ylabel('y axis')
                        axes[1,0].set_title('z plane')
                        axes[1,0].set_aspect('equal')
                        axes[1,1].axis('off')
                        fig.suptitle(f'3D MRI scan with bounding box (Label: {label})')
                        fig.tight_layout()
                        fig.savefig(f'mri_scan_{label}.png')  # Save the figure to a file
                        plt.close(fig)

                        output_dir = os.path.join(drop_dir, 'VeryFinalRepresentationA')
                        os.makedirs(output_dir, exist_ok=True)

                        fig.savefig(os.path.join(output_dir, f'{subj_id}mri_scan_{label}.png'))


100%|██████████| 157/157 [05:31<00:00,  2.11s/it]


## very final B

In [602]:
drop_dir = '/data/projects/TMOR/data/'
seg_dir = '/data/projects/TMOR/data/Deeplearning/'
#indices = []
from tqdm import tqdm
for subj_id in tqdm(os.listdir(seg_dir)):
    #extrasct the correct id from the subjects
    # Define paths to the segmentation and regular t1 postprocessed files for this subject
    seg_glob = os.path.join(seg_dir, f'{subj_id}', f'seg.nii.gz')
    reg_glob = os.path.join(seg_dir, f'{subj_id}', f't1_gd.nii.gz')
    reg2_glob = os.path.join(seg_dir, f'{subj_id}', f't1_pre.nii.gz')
    seg_paths = glob.glob(seg_glob)
    reg_paths = glob.glob(reg_glob)
    reg2_paths = glob.glob(reg2_glob)
    # Loop through all matching segmentation and registration files
    for seg, reg, reg2 in zip(seg_paths, reg_paths, reg2_paths):
        if os.path.exists(seg) and os.path.exists(reg) and os.path.exists(reg2):
            mask = nib.load(seg)
            vol_mask = mask.get_fdata()
            t1_gd = nib.load(reg)
            vol_t1_gd = t1_gd.get_fdata()
            t1_pre = nib.load(reg2)
            vol_t1_pre = t1_pre.get_fdata()
            vol = vol_t1_gd
            for label in np.unique(vol_mask):
                if label != 0:
                    label_indices = np.argwhere(vol_mask == label)
                    if len(label_indices) > 125:
                        indices = label_indices
                        X_min, X_max = np.min(indices[:, 0]), np.max(indices[:, 0])
                        Y_min, Y_max = np.min(indices[:, 1]), np.max(indices[:, 1])
                        Z_min, Z_max = np.min(indices[:, 2]), np.max(indices[:, 2])

                        X_avg = (X_min + X_max) // 2
                        Y_avg = (Y_min + Y_max) // 2
                        Z_avg = (Z_min + Z_max) // 2
                        # Pad the indices with zeros

                        X_low = max(X_avg - (D_x_max // 2), min(np.argwhere(vol_mask != 0)[:, 0]) - 10)
                        X_high = min(X_avg + (D_x_max // 2),  max(np.argwhere(vol_mask != 0)[:, 0]) + 10)
                        Y_low = max(Y_avg - (D_y_max // 2), min(np.argwhere(vol_mask != 0)[:, 1]) - 10)
                        Y_high = min(Y_avg + (D_y_max // 2), max(np.argwhere(vol_mask != 0)[:, 1]) + 10)
                        Z_low = max(Z_avg - (D_z_max // 2), min(np.argwhere(vol_mask != 0)[:, 2]) - 10)
                        Z_high = min(Z_avg + (D_z_max // 2), max(np.argwhere(vol_mask != 0)[:, 2]) + 10)

                        pene = vol_mask[X_low:X_high, Y_low:Y_high, Z_low:Z_high]
                        t1_desp = vol_t1_gd[X_low:X_high, Y_low:Y_high, Z_low:Z_high]
                        t1_antes = vol_t1_pre[X_low:X_high, Y_low:Y_high, Z_low:Z_high]

                        ## define the pads
                        pad_width = [
                            (0, max(D_x_max - pene.shape[0], 0)),
                            (0, max(D_y_max - pene.shape[1], 0)),
                            (0, max(D_z_max - pene.shape[2], 0))
                        ]
                        # define the partitions
                        seg_partitioned = np.pad(pene, pad_width, mode='constant', constant_values=0)
                        t1_gd_partitioned = np.pad(t1_desp, pad_width, mode='constant', constant_values=0)
                        t1_pre_partitioned = np.pad(t1_antes, pad_width, mode='constant', constant_values=0)
                        #
                        output_dir = os.path.join(drop_dir, 'VeryFinalB', subj_id, str(int(label)))
                        os.makedirs(output_dir, exist_ok=True)
                        # tumors segemented of the segmentation of mask
                        output_path_1 = os.path.join(output_dir, 'seg.nii.gz')
                        nib.save(nib.Nifti1Image(seg_partitioned, mask.affine), output_path_1)
                        # tumors segmented of the t1_gd
                        output_path_2 = os.path.join(output_dir, 't1_gd.nii.gz')
                        nib.save(nib.Nifti1Image(t1_gd_partitioned, t1_gd.affine), output_path_2)
                        # tumors segemented if t1_pre
                        output_path_3 = os.path.join(output_dir, 't1_pre.nii.gz')
                        nib.save(nib.Nifti1Image(t1_pre_partitioned, t1_pre.affine), output_path_3)

100%|██████████| 157/157 [02:16<00:00,  1.15it/s]


In [601]:
drop_dir = '/data/projects/TMOR/data/'
seg_dir = '/data/projects/TMOR/data/Deeplearning/'
#indices = []
from tqdm import tqdm
for subj_id in tqdm(os.listdir(seg_dir)):
    #extrasct the correct id from the subjects
    # Define paths to the segmentation and regular t1 postprocessed files for this subject
    seg_glob = os.path.join(seg_dir, f'{subj_id}', f'seg.nii.gz')
    reg_glob = os.path.join(seg_dir, f'{subj_id}', f't1_gd.nii.gz')
    reg2_glob = os.path.join(seg_dir, f'{subj_id}', f't1_pre.nii.gz')
    seg_paths = glob.glob(seg_glob)
    reg_paths = glob.glob(reg_glob)
    reg2_paths = glob.glob(reg2_glob)
    # Loop through all matching segmentation and registration files
    for seg, reg, reg2 in zip(seg_paths, reg_paths, reg2_paths):
        if os.path.exists(seg) and os.path.exists(reg) and os.path.exists(reg2):
            mask = nib.load(seg)
            vol_mask = mask.get_fdata()
            t1_gd = nib.load(reg)
            vol_t1_gd = t1_gd.get_fdata()
            t1_pre = nib.load(reg2)
            vol_t1_pre = t1_pre.get_fdata()
            vol = vol_t1_gd
            for label in np.unique(vol_mask):
                if label != 0:
                    label_indices = np.argwhere(vol_mask == label)
                    if len(label_indices) > 125:
                        indices = label_indices
                        X_min, X_max = np.min(indices[:, 0]), np.max(indices[:, 0])
                        Y_min, Y_max = np.min(indices[:, 1]), np.max(indices[:, 1])
                        Z_min, Z_max = np.min(indices[:, 2]), np.max(indices[:, 2])

                        mean_x = (X_min + X_max) // 2
                        mean_y = (Y_min + Y_max) // 2
                        mean_z = (Z_min + Z_max) // 2
                        # Pad the indices with zeros

                        x_start = max(mean_x - (D_x_max // 2), min(np.argwhere(vol_mask != 0)[:, 0]) - 10)
                        x_end = min(mean_x + (D_x_max // 2),  max(np.argwhere(vol_mask != 0)[:, 0]) + 10)
                        y_start = max(mean_y - (D_y_max // 2), min(np.argwhere(vol_mask != 0)[:, 1]) - 10)
                        y_end = min(mean_y + (D_y_max // 2), max(np.argwhere(vol_mask != 0)[:, 1]) + 10)
                        z_start = max(mean_z - (D_z_max // 2), min(np.argwhere(vol_mask != 0)[:, 2]) - 10)
                        z_end = min(mean_z + (D_z_max // 2), max(np.argwhere(vol_mask != 0)[:, 2]) + 10)

                        fig, axes = plt.subplots(2,2)
                        axes[0,1].imshow(vol[mean_x, :, :].T, origin='lower')
                        axes[0,1].scatter(mean_y, mean_z)
                        axes[0,1].plot([y_start, y_start], [z_start, z_end])
                        axes[0,1].plot([y_end, y_end], [z_start, z_end])
                        axes[0,1].plot([y_start, y_end], [z_start, z_start])
                        axes[0,1].plot([y_start, y_end], [z_end, z_end])
                        axes[0,1].set_xlabel('y axis')
                        axes[0,1].set_ylabel('z axis')
                        axes[0,1].set_title('x plane')
                        axes[0,1].set_aspect('equal')
                        axes[0,0].imshow(vol[:, mean_y, :].T, origin='lower')
                        axes[0,0].scatter(mean_x, mean_z)
                        axes[0,0].plot([x_start, x_start], [z_start, z_end])
                        axes[0,0].plot([x_end, x_end], [z_start, z_end])
                        axes[0,0].plot([x_start, x_end], [z_start, z_start])
                        axes[0,0].plot([x_start, x_end], [z_end, z_end])
                        axes[0,0].set_xlabel('x axis')
                        axes[0,0].set_ylabel('z axis')
                        axes[0,0].set_title('y plane')
                        axes[0,0].set_aspect('equal')
                        axes[1,0].imshow(vol[:, :, mean_z].T, origin='lower')
                        axes[1,0].scatter(mean_x, mean_y)
                        axes[1,0].plot([x_start, x_start], [y_start, y_end])
                        axes[1,0].plot([x_end, x_end], [y_start, y_end])
                        axes[1,0].plot([x_start, x_end], [y_start, y_start])
                        axes[1,0].plot([x_start, x_end], [y_end, y_end])
                        axes[1,0].set_xlabel('x axis')
                        axes[1,0].set_ylabel('y axis')
                        axes[1,0].set_title('z plane')
                        axes[1,0].set_aspect('equal')
                        axes[1,1].axis('off')
                        fig.suptitle(f'3D MRI scan with bounding box (Label: {label})')
                        fig.tight_layout()
                        fig.savefig(f'mri_scan_{label}.png')  # Save the figure to a file
                        plt.close(fig)

                        output_dir = os.path.join(drop_dir, 'VeryFinalRepresentationB')
                        os.makedirs(output_dir, exist_ok=True)

                        fig.savefig(os.path.join(output_dir, f'{subj_id}mri_scan_{label}.png'))



100%|██████████| 157/157 [06:04<00:00,  2.32s/it]


## vERY FINAL C

In [604]:
from matplotlib import pyplot as plt

drop_dir = '/data/projects/TMOR/data/'
seg_dir = '/data/projects/TMOR/data/Deeplearning/'
#indices = []
from tqdm import tqdm
for subj_id in tqdm(os.listdir(seg_dir)):
    #extrasct the correct id from the subjects
    # Define paths to the segmentation and regular t1 postprocessed files for this subject
    seg_glob = os.path.join(seg_dir, f'{subj_id}', f'seg.nii.gz')
    reg_glob = os.path.join(seg_dir, f'{subj_id}', f't1_gd.nii.gz')
    reg2_glob = os.path.join(seg_dir, f'{subj_id}', f't1_pre.nii.gz')
    seg_paths = glob.glob(seg_glob)
    reg_paths = glob.glob(reg_glob)
    reg2_paths = glob.glob(reg2_glob)
    # Loop through all matching segmentation and registration files
    for seg, reg, reg2 in zip(seg_paths, reg_paths, reg2_paths):
        if os.path.exists(seg) and os.path.exists(reg) and os.path.exists(reg2):
            mask = nib.load(seg)
            vol_mask = mask.get_fdata()
            t1_gd = nib.load(reg)
            vol_t1_gd = t1_gd.get_fdata()
            t1_pre = nib.load(reg2)
            vol_t1_pre = t1_pre.get_fdata()
            vol = vol_t1_gd
            for label in np.unique(vol_mask):
                if label != 0:
                    label_indices = np.argwhere(vol_mask == label)
                    if len(label_indices) > 125:
                        indices = label_indices
                        X_min, X_max = np.min(indices[:, 0]), np.max(indices[:, 0])
                        Y_min, Y_max = np.min(indices[:, 1]), np.max(indices[:, 1])
                        Z_min, Z_max = np.min(indices[:, 2]), np.max(indices[:, 2])

                        mean_x = (X_min + X_max) // 2
                        mean_y = (Y_min + Y_max) // 2
                        mean_z  = (Z_min + Z_max) // 2
                        # Pad the indices with zeros
                        x_start1 = max(mean_x - (D_x_max // 2), min(np.argwhere(vol_mask != 0)[:, 0]))
                        x_end1 = min(mean_x + (D_x_max // 2),  max(np.argwhere(vol_mask != 0)[:, 0]))
                        y_start1 = max(mean_y - (D_y_max // 2), min(np.argwhere(vol_mask != 0)[:, 1]))
                        y_end1 = min(mean_y + (D_y_max // 2), max(np.argwhere(vol_mask != 0)[:, 1]))
                        z_start1 = max(mean_z - (D_z_max // 2), min(np.argwhere(vol_mask != 0)[:, 2]))
                        z_end1 = min(mean_z + (D_z_max // 2), max(np.argwhere(vol_mask != 0)[:, 2]))

                        x_diff_down1 = mean_x - x_start1
                        x_diff_up1 = x_end1 - mean_x
                        y_diff_down1 = mean_y - y_start1
                        y_diff_up1 = y_end1 - mean_y
                        z_diff_down1 = mean_z - z_start1
                        z_diff_up1 = z_end1- mean_z########

                        x_diff_down = x_diff_down1 if x_diff_down1 < 20 else 20
                        x_diff_up = x_diff_up1 if x_diff_up1 < 20 else 20
                        y_diff_down = y_diff_down1 if y_diff_down1 < 20 else 20
                        y_diff_up = y_diff_up1 if y_diff_up1 < 20 else 20
                        z_diff_down = z_diff_down1 if z_diff_down1 < 20 else 20
                        z_diff_up = z_diff_up1 if z_diff_up1 < 20 else 20

                        x_start = x_start1 - 5 if x_diff_down < x_diff_up else mean_x - (x_diff_up) - 5
                        x_end = x_end1 + 5 if x_diff_up < x_diff_down else mean_x + (x_diff_down) + 5
                        y_start = y_start1 - 5 if y_diff_down < y_diff_up else mean_y - (y_diff_up) - 5
                        y_end = y_end1 + 5 if y_diff_up < y_diff_down else mean_y + (y_diff_down ) + 5
                        z_start = z_start1 -5 if z_diff_down < z_diff_up else mean_z - (z_diff_up) - 5
                        z_end = z_end1 +5 if z_diff_up < z_diff_down else mean_z + (z_diff_down) + 5

                        #x_start = x_start2 - 5
                        #x_end = x_end2 + 5
                        #y_start = y_start2 - 5
                        #y_end = y_end2 + 5
                        #z_start = z_start2 - 5
                        #z_end =  z_end2 + 5
                        #if mean_x >= vol.shape[0]:
                            #continue
                        pene = vol_mask[x_start:x_end, y_start:y_end, z_start:z_end]
                        t1_desp = vol_t1_gd[x_start:x_end, y_start:y_end, z_start:z_end]
                        t1_antes = vol_t1_pre[x_start:x_end, y_start:y_end, z_start:z_end]

                        ## define the pads
                        pad_width = [
                            (0, max(D_x_max - pene.shape[0], 0)),
                            (0, max(D_y_max - pene.shape[1], 0)),
                            (0, max(D_z_max - pene.shape[2], 0))
                        ]
                        # define the partitions
                        seg_partitioned = np.pad(pene, pad_width, mode='constant', constant_values=0)
                        t1_gd_partitioned = np.pad(t1_desp, pad_width, mode='constant', constant_values=0)
                        t1_pre_partitioned = np.pad(t1_antes, pad_width, mode='constant', constant_values=0)
                        #
                        output_dir = os.path.join(drop_dir, 'VeryFinalC', subj_id, str(int(label)))
                        os.makedirs(output_dir, exist_ok=True)
                        # tumors segemented of the segmentation of mask
                        output_path_1 = os.path.join(output_dir, 'seg.nii.gz')
                        nib.save(nib.Nifti1Image(seg_partitioned, mask.affine), output_path_1)
                        # tumors segmented of the t1_gd
                        output_path_2 = os.path.join(output_dir, 't1_gd.nii.gz')
                        nib.save(nib.Nifti1Image(t1_gd_partitioned, t1_gd.affine), output_path_2)
                        # tumors segemented if t1_pre
                        output_path_3 = os.path.join(output_dir, 't1_pre.nii.gz')
                        nib.save(nib.Nifti1Image(t1_pre_partitioned, t1_pre.affine), output_path_3)


100%|██████████| 157/157 [02:21<00:00,  1.11it/s]


In [605]:
from matplotlib import pyplot as plt

drop_dir = '/data/projects/TMOR/data/'
seg_dir = '/data/projects/TMOR/data/Deeplearning/'
#indices = []
from tqdm import tqdm
for subj_id in tqdm(os.listdir(seg_dir)):
    #extrasct the correct id from the subjects
    # Define paths to the segmentation and regular t1 postprocessed files for this subject
    seg_glob = os.path.join(seg_dir, f'{subj_id}', f'seg.nii.gz')
    reg_glob = os.path.join(seg_dir, f'{subj_id}', f't1_gd.nii.gz')
    reg2_glob = os.path.join(seg_dir, f'{subj_id}', f't1_pre.nii.gz')
    seg_paths = glob.glob(seg_glob)
    reg_paths = glob.glob(reg_glob)
    reg2_paths = glob.glob(reg2_glob)
    # Loop through all matching segmentation and registration files
    for seg, reg, reg2 in zip(seg_paths, reg_paths, reg2_paths):
        if os.path.exists(seg) and os.path.exists(reg) and os.path.exists(reg2):
            mask = nib.load(seg)
            vol_mask = mask.get_fdata()
            t1_gd = nib.load(reg)
            vol_t1_gd = t1_gd.get_fdata()
            t1_pre = nib.load(reg2)
            vol_t1_pre = t1_pre.get_fdata()
            vol = vol_t1_gd
            for label in np.unique(vol_mask):
                if label != 0:
                    label_indices = np.argwhere(vol_mask == label)
                    if len(label_indices) > 125:
                        indices = label_indices
                        X_min, X_max = np.min(indices[:, 0]), np.max(indices[:, 0])
                        Y_min, Y_max = np.min(indices[:, 1]), np.max(indices[:, 1])
                        Z_min, Z_max = np.min(indices[:, 2]), np.max(indices[:, 2])

                        mean_x = (X_min + X_max) // 2
                        mean_y = (Y_min + Y_max) // 2
                        mean_z  = (Z_min + Z_max) // 2
                        # Pad the indices with zeros
                        x_start1 = max(mean_x - (D_x_max // 2), min(np.argwhere(vol_mask != 0)[:, 0]))
                        x_end1 = min(mean_x + (D_x_max // 2),  max(np.argwhere(vol_mask != 0)[:, 0]))
                        y_start1 = max(mean_y - (D_y_max // 2), min(np.argwhere(vol_mask != 0)[:, 1]))
                        y_end1 = min(mean_y + (D_y_max // 2), max(np.argwhere(vol_mask != 0)[:, 1]))
                        z_start1 = max(mean_z - (D_z_max // 2), min(np.argwhere(vol_mask != 0)[:, 2]))
                        z_end1 = min(mean_z + (D_z_max // 2), max(np.argwhere(vol_mask != 0)[:, 2]))

                        x_diff_down1 = mean_x - x_start1
                        x_diff_up1 = x_end1 - mean_x
                        y_diff_down1 = mean_y - y_start1
                        y_diff_up1 = y_end1 - mean_y
                        z_diff_down1 = mean_z - z_start1
                        z_diff_up1 = z_end1- mean_z########

                        x_diff_down = x_diff_down1 if x_diff_down1 < 20 else 20
                        x_diff_up = x_diff_up1 if x_diff_up1 < 20 else 20
                        y_diff_down = y_diff_down1 if y_diff_down1 < 20 else 20
                        y_diff_up = y_diff_up1 if y_diff_up1 < 20 else 20
                        z_diff_down = z_diff_down1 if z_diff_down1 < 20 else 20
                        z_diff_up = z_diff_up1 if z_diff_up1 < 20 else 20

                        x_start = x_start1 - 5 if x_diff_down < x_diff_up else mean_x - (x_diff_up) - 5
                        x_end = x_end1 + 5 if x_diff_up < x_diff_down else mean_x + (x_diff_down) + 5
                        y_start = y_start1 - 5 if y_diff_down < y_diff_up else mean_y - (y_diff_up) - 5
                        y_end = y_end1 + 5 if y_diff_up < y_diff_down else mean_y + (y_diff_down ) + 5
                        z_start = z_start1 -5 if z_diff_down < z_diff_up else mean_z - (z_diff_up) - 5
                        z_end = z_end1 +5 if z_diff_up < z_diff_down else mean_z + (z_diff_down) + 5

                        #x_start = x_start2 - 5
                        #x_end = x_end2 + 5
                        #y_start = y_start2 - 5
                        #y_end = y_end2 + 5
                        #z_start = z_start2 - 5
                        #z_end =  z_end2 + 5
                        #if mean_x >= vol.shape[0]:
                            #continue
                        # Display the tensor
                        fig, axes = plt.subplots(2,2)
                        axes[0,1].imshow(vol[mean_x, :, :].T, origin='lower')
                        axes[0,1].scatter(mean_y, mean_z)
                        axes[0,1].plot([y_start, y_start], [z_start, z_end])
                        axes[0,1].plot([y_end, y_end], [z_start, z_end])
                        axes[0,1].plot([y_start, y_end], [z_start, z_start])
                        axes[0,1].plot([y_start, y_end], [z_end, z_end])
                        axes[0,1].set_xlabel('y axis')
                        axes[0,1].set_ylabel('z axis')
                        axes[0,1].set_title('x plane')
                        axes[0,1].set_aspect('equal')
                        axes[0,0].imshow(vol[:, mean_y, :].T, origin='lower')
                        axes[0,0].scatter(mean_x, mean_z)
                        axes[0,0].plot([x_start, x_start], [z_start, z_end])
                        axes[0,0].plot([x_end, x_end], [z_start, z_end])
                        axes[0,0].plot([x_start, x_end], [z_start, z_start])
                        axes[0,0].plot([x_start, x_end], [z_end, z_end])
                        axes[0,0].set_xlabel('x axis')
                        axes[0,0].set_ylabel('z axis')
                        axes[0,0].set_title('y plane')
                        axes[0,0].set_aspect('equal')
                        axes[1,0].imshow(vol[:, :, mean_z].T, origin='lower')
                        axes[1,0].scatter(mean_x, mean_y)
                        axes[1,0].plot([x_start, x_start], [y_start, y_end])
                        axes[1,0].plot([x_end, x_end], [y_start, y_end])
                        axes[1,0].plot([x_start, x_end], [y_start, y_start])
                        axes[1,0].plot([x_start, x_end], [y_end, y_end])
                        axes[1,0].set_xlabel('x axis')
                        axes[1,0].set_ylabel('y axis')
                        axes[1,0].set_title('z plane')
                        axes[1,0].set_aspect('equal')
                        axes[1,1].axis('off')
                        fig.suptitle(f'3D MRI scan with bounding box (Label: {label})')
                        fig.tight_layout()
                        fig.savefig(f'mri_scan_{label}.png')  # Save the figure to a file
                        plt.close(fig)

                        output_dir = os.path.join(drop_dir, 'VeryFinalRepresentationC')
                        os.makedirs(output_dir, exist_ok=True)

                        fig.savefig(os.path.join(output_dir, f'{subj_id}mri_scan_{label}.png'))


100%|██████████| 157/157 [06:32<00:00,  2.50s/it]


## Very final representation D

In [608]:
from matplotlib import pyplot as plt

drop_dir = '/data/projects/TMOR/data/'
seg_dir = '/data/projects/TMOR/data/Deeplearning/'
#indices = []
from tqdm import tqdm
for subj_id in tqdm(os.listdir(seg_dir)):
    #extrasct the correct id from the subjects
    # Define paths to the segmentation and regular t1 postprocessed files for this subject
    seg_glob = os.path.join(seg_dir, f'{subj_id}', f'seg.nii.gz')
    reg_glob = os.path.join(seg_dir, f'{subj_id}', f't1_gd.nii.gz')
    reg2_glob = os.path.join(seg_dir, f'{subj_id}', f't1_pre.nii.gz')
    seg_paths = glob.glob(seg_glob)
    reg_paths = glob.glob(reg_glob)
    reg2_paths = glob.glob(reg2_glob)
    # Loop through all matching segmentation and registration files
    for seg, reg, reg2 in zip(seg_paths, reg_paths, reg2_paths):
        if os.path.exists(seg) and os.path.exists(reg) and os.path.exists(reg2):
            mask = nib.load(seg)
            vol_mask = mask.get_fdata()
            t1_gd = nib.load(reg)
            vol_t1_gd = t1_gd.get_fdata()
            t1_pre = nib.load(reg2)
            vol_t1_pre = t1_pre.get_fdata()
            vol = vol_t1_gd
            for label in np.unique(vol_mask):
                if label != 0:
                    label_indices = np.argwhere(vol_mask == label)
                    if len(label_indices) > 125:
                        indices = label_indices
                        X_min, X_max = np.min(indices[:, 0]), np.max(indices[:, 0])
                        Y_min, Y_max = np.min(indices[:, 1]), np.max(indices[:, 1])
                        Z_min, Z_max = np.min(indices[:, 2]), np.max(indices[:, 2])

                        mean_x = (X_min + X_max) // 2
                        mean_y = (Y_min + Y_max) // 2
                        mean_z  = (Z_min + Z_max) // 2
                        # Pad the indices with zeros
                        x_start = X_min - 15
                        x_end = X_max + 15
                        y_start = Y_min - 15
                        y_end = Y_max + 15
                        z_start = Z_min - 15
                        z_end = Z_max + 15

                        pene = vol_mask[x_start:x_end, y_start:y_end, z_start:z_end]
                        t1_desp = vol_t1_gd[x_start:x_end, y_start:y_end, z_start:z_end]
                        t1_antes = vol_t1_pre[x_start:x_end, y_start:y_end, z_start:z_end]

                        ## define the pads
                        pad_width = [
                            (0, max(D_x_max - pene.shape[0], 0)),
                            (0, max(D_y_max - pene.shape[1], 0)),
                            (0, max(D_z_max - pene.shape[2], 0))
                        ]
                        # define the partitions
                        seg_partitioned = np.pad(pene, pad_width, mode='constant', constant_values=0)
                        t1_gd_partitioned = np.pad(t1_desp, pad_width, mode='constant', constant_values=0)
                        t1_pre_partitioned = np.pad(t1_antes, pad_width, mode='constant', constant_values=0)
                        #
                        output_dir = os.path.join(drop_dir, 'VeryFinalD', subj_id, str(int(label)))
                        os.makedirs(output_dir, exist_ok=True)
                        # tumors segemented of the segmentation of mask
                        output_path_1 = os.path.join(output_dir, 'seg.nii.gz')
                        nib.save(nib.Nifti1Image(seg_partitioned, mask.affine), output_path_1)
                        # tumors segmented of the t1_gd
                        output_path_2 = os.path.join(output_dir, 't1_gd.nii.gz')
                        nib.save(nib.Nifti1Image(t1_gd_partitioned, t1_gd.affine), output_path_2)
                        # tumors segemented if t1_pre
                        output_path_3 = os.path.join(output_dir, 't1_pre.nii.gz')
                        nib.save(nib.Nifti1Image(t1_pre_partitioned, t1_pre.affine), output_path_3)


100%|██████████| 157/157 [01:39<00:00,  1.57it/s]


In [607]:
from matplotlib import pyplot as plt

drop_dir = '/data/projects/TMOR/data/'
seg_dir = '/data/projects/TMOR/data/Deeplearning/'
#indices = []
from tqdm import tqdm
for subj_id in tqdm(os.listdir(seg_dir)):
    #extrasct the correct id from the subjects
    # Define paths to the segmentation and regular t1 postprocessed files for this subject
    seg_glob = os.path.join(seg_dir, f'{subj_id}', f'seg.nii.gz')
    reg_glob = os.path.join(seg_dir, f'{subj_id}', f't1_gd.nii.gz')
    reg2_glob = os.path.join(seg_dir, f'{subj_id}', f't1_pre.nii.gz')
    seg_paths = glob.glob(seg_glob)
    reg_paths = glob.glob(reg_glob)
    reg2_paths = glob.glob(reg2_glob)
    # Loop through all matching segmentation and registration files
    for seg, reg, reg2 in zip(seg_paths, reg_paths, reg2_paths):
        if os.path.exists(seg) and os.path.exists(reg) and os.path.exists(reg2):
            mask = nib.load(seg)
            vol_mask = mask.get_fdata()
            t1_gd = nib.load(reg)
            vol_t1_gd = t1_gd.get_fdata()
            t1_pre = nib.load(reg2)
            vol_t1_pre = t1_pre.get_fdata()
            vol = vol_t1_gd
            for label in np.unique(vol_mask):
                if label != 0:
                    label_indices = np.argwhere(vol_mask == label)
                    if len(label_indices) > 125:
                        indices = label_indices
                        X_min, X_max = np.min(indices[:, 0]), np.max(indices[:, 0])
                        Y_min, Y_max = np.min(indices[:, 1]), np.max(indices[:, 1])
                        Z_min, Z_max = np.min(indices[:, 2]), np.max(indices[:, 2])

                        mean_x = (X_min + X_max) // 2
                        mean_y = (Y_min + Y_max) // 2
                        mean_z  = (Z_min + Z_max) // 2
                        # Pad the indices with zeros
                        x_start = X_min - 15
                        x_end = X_max + 15
                        y_start = Y_min - 15
                        y_end = Y_max + 15
                        z_start = Z_min - 15
                        z_end = Z_max + 15

                        # Display the tensor
                        fig, axes = plt.subplots(2,2)
                        axes[0,1].imshow(vol[mean_x, :, :].T, origin='lower')
                        axes[0,1].scatter(mean_y, mean_z)
                        axes[0,1].plot([y_start, y_start], [z_start, z_end])
                        axes[0,1].plot([y_end, y_end], [z_start, z_end])
                        axes[0,1].plot([y_start, y_end], [z_start, z_start])
                        axes[0,1].plot([y_start, y_end], [z_end, z_end])
                        axes[0,1].set_xlabel('y axis')
                        axes[0,1].set_ylabel('z axis')
                        axes[0,1].set_title('x plane')
                        axes[0,1].set_aspect('equal')
                        axes[0,0].imshow(vol[:, mean_y, :].T, origin='lower')
                        axes[0,0].scatter(mean_x, mean_z)
                        axes[0,0].plot([x_start, x_start], [z_start, z_end])
                        axes[0,0].plot([x_end, x_end], [z_start, z_end])
                        axes[0,0].plot([x_start, x_end], [z_start, z_start])
                        axes[0,0].plot([x_start, x_end], [z_end, z_end])
                        axes[0,0].set_xlabel('x axis')
                        axes[0,0].set_ylabel('z axis')
                        axes[0,0].set_title('y plane')
                        axes[0,0].set_aspect('equal')
                        axes[1,0].imshow(vol[:, :, mean_z].T, origin='lower')
                        axes[1,0].scatter(mean_x, mean_y)
                        axes[1,0].plot([x_start, x_start], [y_start, y_end])
                        axes[1,0].plot([x_end, x_end], [y_start, y_end])
                        axes[1,0].plot([x_start, x_end], [y_start, y_start])
                        axes[1,0].plot([x_start, x_end], [y_end, y_end])
                        axes[1,0].set_xlabel('x axis')
                        axes[1,0].set_ylabel('y axis')
                        axes[1,0].set_title('z plane')
                        axes[1,0].set_aspect('equal')
                        axes[1,1].axis('off')
                        fig.suptitle(f'3D MRI scan with bounding box (Label: {label})')
                        fig.tight_layout()
                        fig.savefig(f'mri_scan_{label}.png')  # Save the figure to a file
                        plt.close(fig)

                        output_dir = os.path.join(drop_dir, 'VeryFinalRepresentationD')
                        os.makedirs(output_dir, exist_ok=True)

                        fig.savefig(os.path.join(output_dir, f'{subj_id}mri_scan_{label}.png'))


100%|██████████| 157/157 [05:56<00:00,  2.27s/it]


## nothing more this is maybe a E but dont thin so

In [451]:
from matplotlib import pyplot as plt

drop_dir = '/data/projects/TMOR/data/'
seg_dir = '/data/projects/TMOR/data/Deeplearning/'
#indices = []
from tqdm import tqdm
for subj_id in tqdm(os.listdir(seg_dir)):
    if subj_id != 'Oslo66':
        #extrasct the correct id from the subjects
        # Define paths to the segmentation and regular t1 postprocessed files for this subject
        seg_glob = os.path.join(seg_dir, f'{subj_id}', f'seg.nii.gz')
        reg_glob = os.path.join(seg_dir, f'{subj_id}', f't1_gd.nii.gz')
        reg2_glob = os.path.join(seg_dir, f'{subj_id}', f't1_pre.nii.gz')
        seg_paths = glob.glob(seg_glob)
        reg_paths = glob.glob(reg_glob)
        reg2_paths = glob.glob(reg2_glob)
        # Loop through all matching segmentation and registration files
        for seg, reg, reg2 in zip(seg_paths, reg_paths, reg2_paths):
            if os.path.exists(seg) and os.path.exists(reg) and os.path.exists(reg2):
                mask = nib.load(seg)
                vol_mask = mask.get_fdata()
                t1_gd = nib.load(reg)
                vol_t1_gd = t1_gd.get_fdata()
                t1_pre = nib.load(reg2)
                vol_t1_pre = t1_pre.get_fdata()
                vol = vol_t1_gd
                for label in np.unique(vol_mask):
                    if label != 0:
                        label_indices = np.argwhere(vol_mask == label)
                        if len(label_indices) > 125:
                            indices = label_indices
                            X_min, X_max = np.min(indices[:, 0]), np.max(indices[:, 0])
                            Y_min, Y_max = np.min(indices[:, 1]), np.max(indices[:, 1])
                            Z_min, Z_max = np.min(indices[:, 2]), np.max(indices[:, 2])

                            mean_x = (X_min + X_max) // 2
                            mean_y = (Y_min + Y_max) // 2
                            mean_z  = (Z_min + Z_max) // 2
                            # Pad the indices with zeros
                            x_start = max(mean_x - (D_x_max // 2), min(np.argwhere(vol_mask != 0)[:, 0]) - 10)
                            x_end = min(mean_x + (D_x_max // 2),  max(np.argwhere(vol_mask != 0)[:, 0]) + 10)
                            y_start = max(mean_y - (D_y_max // 2), min(np.argwhere(vol_mask != 0)[:, 1]) - 10)
                            y_end = min(mean_y + (D_y_max // 2), max(np.argwhere(vol_mask != 0)[:, 1]) + 10)
                            z_start = max(mean_z - (D_z_max // 2), min(np.argwhere(vol_mask != 0)[:, 2]) - 10)
                            z_end = min(mean_z + (D_z_max // 2), max(np.argwhere(vol_mask != 0)[:, 2]) + 10)
                            #if mean_x >= vol.shape[0]:
                                #continue
                            # Display the tensor
                            fig, axes = plt.subplots(2,2)
                            axes[0,0].imshow(vol[mean_x])
                            axes[0,0].scatter(mean_z, mean_y)
                            axes[0,0].plot([z_start, z_end], [y_start, y_start])
                            axes[0,0].plot([z_start, z_end], [y_end, y_end])
                            axes[0,0].plot([z_start, z_start], [y_start, y_end])
                            axes[0,0].plot([z_end, z_end], [y_start, y_end])
                            axes[0,0].set_xlabel('y axis')
                            axes[0,0].set_ylabel('z axis')
                            axes[0,0].set_title('x plane')
                            axes[0,0].set_aspect('equal')
                            axes[0,1].imshow(vol[:, mean_y])
                            axes[0,1].scatter(mean_z, mean_x)
                            axes[0,1].plot([z_start, z_end], [x_start, x_start])
                            axes[0,1].plot([z_start, z_end], [x_end, x_end])
                            axes[0,1].plot([z_start, z_start], [x_start, x_end])
                            axes[0,1].plot([z_end, z_end], [x_start, x_end])
                            axes[0,1].set_xlabel('x axis')
                            axes[0,1].set_ylabel('z axis')
                            axes[0,1].set_title('y plane')
                            axes[0,1].set_aspect('equal')
                            axes[1,0].imshow(vol[:, :, mean_z])
                            axes[1,0].scatter(mean_y, mean_x)
                            axes[1,0].plot([y_start, y_start], [x_start, x_end])
                            axes[1,0].plot([y_end, y_end], [x_start, x_end])
                            axes[1,0].plot([y_start, y_end], [x_start, x_start])
                            axes[1,0].plot([y_start, y_end], [x_end, x_end])
                            axes[1,0].set_xlabel('x axis')
                            axes[1,0].set_ylabel('y axis')
                            axes[1,0].set_title('z plane')
                            axes[1,0].set_aspect('equal')
                            axes[1,1].axis('off')
                            fig.suptitle(f'3D MRI scan with bounding box (Label: {label})')
                            fig.tight_layout()
                            fig.savefig(f'mri_scan_{label}.png')  # Save the figure to a file
                            plt.close(fig)

                            output_dir = os.path.join(drop_dir, 'VeryFinalRepresentationD')
                            os.makedirs(output_dir, exist_ok=True)

                            fig.savefig(os.path.join(output_dir, f'{subj_id}mri_scan_{label}.png'))

 17%|█▋        | 27/157 [00:33<02:41,  1.25s/it]


KeyboardInterrupt: 

## Maarten way

In [611]:
drop_dir = '/data/projects/TMOR/data/'

def clamp(num, mn, mx):
    """
    Clamps a number between two values.
    :param num: The number to clamp.
    :param mn: The minimum value.
    :param mx: The maximum value.
    :return: The clamped number.
    """
    return max(min(num, mx), mn)


seg_dir = '/data/projects/TMOR/data/Deeplearning/'
#indices = []
for subj_id in tqdm(os.listdir(seg_dir)):
    #extrasct the correct id from the subjects
    # Define paths to the segmentation and regular t1 postprocessed files for this subject
    seg_glob = os.path.join(seg_dir, f'{subj_id}', f'seg.nii.gz')
    reg_glob = os.path.join(seg_dir, f'{subj_id}', f't1_gd.nii.gz')
    reg2_glob = os.path.join(seg_dir, f'{subj_id}', f't1_pre.nii.gz')
    seg_paths = glob.glob(seg_glob)
    reg_paths = glob.glob(reg_glob)
    reg2_paths = glob.glob(reg2_glob)
    # Loop through all matching segmentation and registration files
    for seg, reg, reg2 in zip(seg_paths, reg_paths, reg2_paths):
        if os.path.exists(seg) and os.path.exists(reg) and os.path.exists(reg2):
            mask = nib.load(seg)
            vol_mask = mask.get_fdata()
            t1_gd = nib.load(reg)
            vol_t1_gd = t1_gd.get_fdata()
            t1_pre = nib.load(reg2)
            vol_t1_pre = t1_pre.get_fdata()
            for label in np.unique(vol_mask):
                if label != 0:
                    label_indices = np.argwhere(vol_mask == label)
                    if len(label_indices) > 125:
                        indices = label_indices
                        X_min, X_max = np.min(indices[:, 0]), np.max(indices[:, 0])
                        Y_min, Y_max = np.min(indices[:, 1]), np.max(indices[:, 1])
                        Z_min, Z_max = np.min(indices[:, 2]), np.max(indices[:, 2])

                        mean_x = (X_min + X_max) // 2
                        mean_y = (Y_min + Y_max) // 2
                        mean_z = (Z_min + Z_max) // 2
                        ###############################
                        d_x = D_x_max
                        d_y = D_y_max
                        d_z = D_z_max
                        ##################################
                        x_start = clamp(mean_x - d_x // 2, 0, vol_mask.shape[0])
                        x_end = clamp(mean_x + d_x // 2, 0, vol_mask.shape[0])

                        y_start = clamp(mean_y - d_y // 2, 0, vol_mask.shape[1])
                        y_end = clamp(mean_y + d_y // 2, 0, vol_mask.shape[1])

                        z_start = clamp(mean_z - d_z // 2, 0, vol_mask.shape[2])
                        z_end = clamp(mean_z + d_z // 2, 0, vol_mask.shape[2])

                        tumor_cutout = vol_mask[x_start:x_end, y_start:y_end, z_start:z_end]
                        t1_gd_cutout = vol_t1_gd[x_start:x_end, y_start:y_end, z_start:z_end]
                        t1_pre_cutout = vol_t1_pre[x_start:x_end, y_start:y_end, z_start:z_end]
                        ###################################
                        tensor = np.zeros([d_x, d_y, d_z])
                        # Define the position of the bounding box in the tensor
                        tx_start = abs(mean_x - d_x // 2) if mean_x - d_x // 2 < 0 else 0
                        tx_end = tumor_cutout.shape[0] + tx_start

                        ty_start = abs(mean_y - d_y // 2) if mean_y - d_y // 2 < 0 else 0
                        ty_end = tumor_cutout.shape[1] + ty_start

                        tz_start = abs(mean_z - d_z // 2) if mean_z - d_z // 2 < 0 else 0
                        tz_end = tumor_cutout.shape[2] + tz_start

                        ## place the output
                        output_dir = os.path.join(drop_dir, 'VeryFinalM', subj_id, str(int(label)))
                        os.makedirs(output_dir, exist_ok=True)
                        # Place the bounding box in the tensor
                        tensor[tx_start:tx_end, ty_start:ty_end, tz_start:tz_end] = tumor_cutout
                         # Save the seg_partitioned into a file
                        #output_path = os.path.join(output_dir, f'{label}seg_seg_.nii.gz')
                        #nib.save(nib.Nifti1Image(tumor_cutout, mask.affine), output_path)
                        output_path_1 = os.path.join(output_dir, 'seg.nii.gz')
                        nib.save(nib.Nifti1Image(tumor_cutout, mask.affine), output_path_1)
                        ### t1_gd
                        tensor[tx_start:tx_end, ty_start:ty_end, tz_start:tz_end] = t1_gd_cutout
                        output_path_2 = os.path.join(output_dir, 't1_gd.nii.gz')
                        nib.save(nib.Nifti1Image(t1_gd_cutout, t1_gd.affine), output_path_2)

                        ### t1_pre
                        tensor[tx_start:tx_end, ty_start:ty_end, tz_start:tz_end] = t1_pre_cutout
                        output_path_3 = os.path.join(output_dir, 't1_pre.nii.gz')
                        nib.save(nib.Nifti1Image(t1_pre_cutout, t1_pre.affine), output_path_3)


100%|██████████| 157/157 [01:35<00:00,  1.64it/s]


In [609]:
drop_dir = '/data/projects/TMOR/data/'

def clamp(num, mn, mx):
    """
    Clamps a number between two values.
    :param num: The number to clamp.
    :param mn: The minimum value.
    :param mx: The maximum value.
    :return: The clamped number.
    """
    return max(min(num, mx), mn)


seg_dir = '/data/projects/TMOR/data/Deeplearning/'
#indices = []
for subj_id in tqdm(os.listdir(seg_dir)):
    #extrasct the correct id from the subjects
    # Define paths to the segmentation and regular t1 postprocessed files for this subject
    seg_glob = os.path.join(seg_dir, f'{subj_id}', f'seg.nii.gz')
    reg_glob = os.path.join(seg_dir, f'{subj_id}', f't1_gd.nii.gz')
    reg2_glob = os.path.join(seg_dir, f'{subj_id}', f't1_pre.nii.gz')
    seg_paths = glob.glob(seg_glob)
    reg_paths = glob.glob(reg_glob)
    reg2_paths = glob.glob(reg2_glob)
    # Loop through all matching segmentation and registration files
    for seg, reg, reg2 in zip(seg_paths, reg_paths, reg2_paths):
        if os.path.exists(seg) and os.path.exists(reg) and os.path.exists(reg2):
            mask = nib.load(seg)
            vol_mask = mask.get_fdata()
            t1_gd = nib.load(reg)
            vol_t1_gd = t1_gd.get_fdata()
            t1_pre = nib.load(reg2)
            vol_t1_pre = t1_pre.get_fdata()
            vol = vol_t1_gd
            for label in np.unique(vol_mask):
                if label != 0:
                    label_indices = np.argwhere(vol_mask == label)
                    if len(label_indices) > 125:
                        indices = label_indices
                        X_min, X_max = np.min(indices[:, 0]), np.max(indices[:, 0])
                        Y_min, Y_max = np.min(indices[:, 1]), np.max(indices[:, 1])
                        Z_min, Z_max = np.min(indices[:, 2]), np.max(indices[:, 2])

                        mean_x = (X_min + X_max) // 2
                        mean_y = (Y_min + Y_max) // 2
                        mean_z = (Z_min + Z_max) // 2
                        ###############################
                        d_x = D_x_max
                        d_y = D_y_max
                        d_z = D_z_max
                        ##################################
                        x_start = clamp(mean_x - d_x // 2, 0, vol_mask.shape[0])
                        x_end = clamp(mean_x + d_x // 2, 0, vol_mask.shape[0])

                        y_start = clamp(mean_y - d_y // 2, 0, vol_mask.shape[1])
                        y_end = clamp(mean_y + d_y // 2, 0, vol_mask.shape[1])

                        z_start = clamp(mean_z - d_z // 2, 0, vol_mask.shape[2])
                        z_end = clamp(mean_z + d_z // 2, 0, vol_mask.shape[2])

                        tumor_cutout = vol_mask[x_start:x_end, y_start:y_end, z_start:z_end]
                        t1_gd_cutout = vol_t1_gd[x_start:x_end, y_start:y_end, z_start:z_end]
                        t1_pre_cutout = vol_t1_pre[x_start:x_end, y_start:y_end, z_start:z_end]
                        ###################################
                        tensor = np.zeros([d_x, d_y, d_z])
                        # Define the position of the bounding box in the tensor
                        tx_start = abs(mean_x - d_x // 2) if mean_x - d_x // 2 < 0 else 0
                        tx_end = tumor_cutout.shape[0] + tx_start

                        ty_start = abs(mean_y - d_y // 2) if mean_y - d_y // 2 < 0 else 0
                        ty_end = tumor_cutout.shape[1] + ty_start

                        tz_start = abs(mean_z - d_z // 2) if mean_z - d_z // 2 < 0 else 0
                        tz_end = tumor_cutout.shape[2] + tz_start

                        fig, axes = plt.subplots(2,2)
                        axes[0,1].imshow(vol[mean_x, :, :].T, origin='lower')
                        axes[0,1].scatter(mean_y, mean_z)
                        axes[0,1].plot([y_start, y_start], [z_start, z_end])
                        axes[0,1].plot([y_end, y_end], [z_start, z_end])
                        axes[0,1].plot([y_start, y_end], [z_start, z_start])
                        axes[0,1].plot([y_start, y_end], [z_end, z_end])
                        axes[0,1].set_xlabel('y axis')
                        axes[0,1].set_ylabel('z axis')
                        axes[0,1].set_title('x plane')
                        axes[0,1].set_aspect('equal')
                        axes[0,0].imshow(vol[:, mean_y, :].T, origin='lower')
                        axes[0,0].scatter(mean_x, mean_z)
                        axes[0,0].plot([x_start, x_start], [z_start, z_end])
                        axes[0,0].plot([x_end, x_end], [z_start, z_end])
                        axes[0,0].plot([x_start, x_end], [z_start, z_start])
                        axes[0,0].plot([x_start, x_end], [z_end, z_end])
                        axes[0,0].set_xlabel('x axis')
                        axes[0,0].set_ylabel('z axis')
                        axes[0,0].set_title('y plane')
                        axes[0,0].set_aspect('equal')
                        axes[1,0].imshow(vol[:, :, mean_z].T, origin='lower')
                        axes[1,0].scatter(mean_x, mean_y)
                        axes[1,0].plot([x_start, x_start], [y_start, y_end])
                        axes[1,0].plot([x_end, x_end], [y_start, y_end])
                        axes[1,0].plot([x_start, x_end], [y_start, y_start])
                        axes[1,0].plot([x_start, x_end], [y_end, y_end])
                        axes[1,0].set_xlabel('x axis')
                        axes[1,0].set_ylabel('y axis')
                        axes[1,0].set_title('z plane')
                        axes[1,0].set_aspect('equal')
                        axes[1,1].axis('off')
                        fig.suptitle(f'3D MRI scan with bounding box (Label: {label})')
                        fig.tight_layout()
                        fig.savefig(f'mri_scan_{label}.png')  # Save the figure to a file
                        plt.close(fig)

                        output_dir = os.path.join(drop_dir, 'VeryFinalRepresentationM')
                        os.makedirs(output_dir, exist_ok=True)

                        fig.savefig(os.path.join(output_dir, f'{subj_id}mri_scan_{label}.png'))

100%|██████████| 157/157 [05:43<00:00,  2.18s/it]


In [156]:
df2 = df[['subjid']].join(df['label']).join(df['labels'])
df2.to_excel('final_labels.xlsx')

In [610]:
D_x_max

86

## standadrd scale the images

In [33]:
import os
import glob
import numpy as np
import nibabel as nib
from sklearn.preprocessing import MinMaxScaler, StandardScaler

def normalize_nifti_images(img_dir, drop_dir, normalization=None):
    indices = glob.glob(os.path.join(img_dir, '*', '*'))
    for i in indices:
        path = os.path.join(i, f't1_gd.nii.gz')
        name_folder = path.split('/')[-4]
        subj_id = path.split('/')[-3]
        label = path.split('/')[-2]
        image  = nib.load(path)
        vol = image.get_fdata()
        #print(vol.shape)

        if normalization == 'standard':
            scaler = MinMaxScaler()
            normalized_image_data = scaler.fit_transform(vol.reshape(-1, vol.shape[-1])).reshape(vol.shape)

        output_dir = os.path.join(drop_dir, f'Sandard_f{name_folder}', subj_id, str(int(label)))
        os.makedirs(output_dir, exist_ok=True)
        # Place the bounding box in the tensor
        output_path_1 = os.path.join(output_dir, 't1_gd.nii.gz')
        nib.save(nib.Nifti1Image(normalized_image_data, image.affine), output_path_1)
        # Example usage:

img_dir = '/data/projects/TMOR/data/VeryFinalA/'
drop_dir =  '/data/projects/TMOR/data/'
#normalize_nifti_images(input_folder_path, output_folder_path, normalization_type='min-max')
normalize_nifti_images(img_dir, drop_dir, normalization='standard')


FileNotFoundError: No such file or no access: '/data/projects/TMOR/data/VeryFinalA/veryfinalA_normalized/Sandard_f/t1_gd.nii.gz'

In [45]:
img_dir = '/data/projects/TMOR/data/Sandard_fVeryFinalA/Oslo65/1/t1_gd.nii.gz'

image  = nib.load(img_dir)

image.ndim

3